In [1]:
spark

# Temporary Table 

In [2]:
from pyspark.sql import SparkSession
spark=SparkSession.builder \
.appName('Table Demo') \
.enableHiveSupport() \
.getOrCreate()

25/04/16 09:36:50 WARN SparkSession: Using an existing Spark session; only runtime SQL configurations will take effect.


In [5]:
df=spark.read \
.format('csv') \
.option('header','True') \
.option('inferSchema','True') \
.load('/data/first_100_customers.csv')

In [6]:
df.show(5)

+-----------+----------+---------+-----------+-------+-----------------+---------+
|customer_id|      name|     city|      state|country|registration_date|is_active|
+-----------+----------+---------+-----------+-------+-----------------+---------+
|          0|Customer_0|     Pune|Maharashtra|  India|       2023-06-29|    false|
|          1|Customer_1|Bangalore| Tamil Nadu|  India|       2023-12-07|     true|
|          2|Customer_2|Hyderabad|    Gujarat|  India|       2023-10-27|     true|
|          3|Customer_3|Bangalore|  Karnataka|  India|       2023-10-17|    false|
|          4|Customer_4|Ahmedabad|  Karnataka|  India|       2023-03-14|    false|
+-----------+----------+---------+-----------+-------+-----------------+---------+
only showing top 5 rows



In [11]:
df.createOrReplaceTempView('temp_customers')

In [13]:
spark.sql('show tables').show()

+---------+--------------+-----------+
|namespace|     tableName|isTemporary|
+---------+--------------+-----------+
|         |temp_customers|       true|
+---------+--------------+-----------+



In [16]:
spark.sql('select * from temp_customers limit 5').show()

+-----------+----------+---------+-----------+-------+-----------------+---------+
|customer_id|      name|     city|      state|country|registration_date|is_active|
+-----------+----------+---------+-----------+-------+-----------------+---------+
|          0|Customer_0|     Pune|Maharashtra|  India|       2023-06-29|    false|
|          1|Customer_1|Bangalore| Tamil Nadu|  India|       2023-12-07|     true|
|          2|Customer_2|Hyderabad|    Gujarat|  India|       2023-10-27|     true|
|          3|Customer_3|Bangalore|  Karnataka|  India|       2023-10-17|    false|
|          4|Customer_4|Ahmedabad|  Karnataka|  India|       2023-03-14|    false|
+-----------+----------+---------+-----------+-------+-----------------+---------+



In [17]:
spark_new=spark.newSession()

In [18]:
spark_new.sql('show tables').show()

+---------+---------+-----------+
|namespace|tableName|isTemporary|
+---------+---------+-----------+
+---------+---------+-----------+



In [ ]:
spark_new.sql('select * from temp_customers limit 5').show()    ## will give error bacuse its not accessible outside of session


# Global Temporary Table

In [72]:
df.createOrReplaceGlobalTempView('global_customeers')

In [73]:
spark.sql('show tables').show()

+---------+--------------+-----------+
|namespace|     tableName|isTemporary|
+---------+--------------+-----------+
|         |temp_customers|       true|
+---------+--------------+-----------+



In [74]:
spark.sql('show tables in global_temp').show()   ### remember it always 

+-----------+-----------------+-----------+
|  namespace|        tableName|isTemporary|
+-----------+-----------------+-----------+
|global_temp|global_customeers|       true|
|global_temp|  temp_customeers|       true|
|           |   temp_customers|       true|
+-----------+-----------------+-----------+



In [75]:
spark.sql("SELECT * FROM global_temp.global_customeers LIMIT 5").show()


+-----------+----------+---------+-----------+-------+-----------------+---------+
|customer_id|      name|     city|      state|country|registration_date|is_active|
+-----------+----------+---------+-----------+-------+-----------------+---------+
|          0|Customer_0|     Pune|Maharashtra|  India|       2023-06-29|    false|
|          1|Customer_1|Bangalore| Tamil Nadu|  India|       2023-12-07|     true|
|          2|Customer_2|Hyderabad|    Gujarat|  India|       2023-10-27|     true|
|          3|Customer_3|Bangalore|  Karnataka|  India|       2023-10-17|    false|
|          4|Customer_4|Ahmedabad|  Karnataka|  India|       2023-03-14|    false|
+-----------+----------+---------+-----------+-------+-----------------+---------+



In [76]:
spark.sql('drop table global_temp.global_customeers')

DataFrame[]

In [ ]:
## Global Tem

In [77]:
spark_new=spark.newSession()

In [78]:
spark_new.sql('show tables').show()

+---------+---------+-----------+
|namespace|tableName|isTemporary|
+---------+---------+-----------+
+---------+---------+-----------+



In [80]:
spark_new.sql('show tables in global_temp').show()

+-----------+---------------+-----------+
|  namespace|      tableName|isTemporary|
+-----------+---------------+-----------+
|global_temp|temp_customeers|       true|
+-----------+---------------+-----------+



In [71]:
spark_new.sql ('select * from global_temp.temp_customeers limit 5').show()

+-----------+----------+---------+-----------+-------+-----------------+---------+
|customer_id|      name|     city|      state|country|registration_date|is_active|
+-----------+----------+---------+-----------+-------+-----------------+---------+
|          0|Customer_0|     Pune|Maharashtra|  India|       2023-06-29|    false|
|          1|Customer_1|Bangalore| Tamil Nadu|  India|       2023-12-07|     true|
|          2|Customer_2|Hyderabad|    Gujarat|  India|       2023-10-27|     true|
|          3|Customer_3|Bangalore|  Karnataka|  India|       2023-10-17|    false|
|          4|Customer_4|Ahmedabad|  Karnataka|  India|       2023-03-14|    false|
+-----------+----------+---------+-----------+-------+-----------------+---------+



## Persistent Table
-- accessible acreoss session

In [82]:
df.write.mode('overwrite').saveAsTable('customers_persistent')

25/04/16 11:29:56 WARN SessionState: METASTORE_FILTER_HOOK will be ignored, since hive.security.authorization.manager is set to instance of HiveAuthorizerFactory.


In [83]:
spark.sql('show tables').show() 

+---------+--------------------+-----------+
|namespace|           tableName|isTemporary|
+---------+--------------------+-----------+
|  default|customers_persistent|      false|
|         |      temp_customers|       true|
+---------+--------------------+-----------+



In [95]:
spark_new_app_session=spark.newSession()

In [96]:
spark_new_app_session.sql('show tables').show()

+---------+--------------------+-----------+
|namespace|           tableName|isTemporary|
+---------+--------------------+-----------+
|  default|customers_persistent|      false|
+---------+--------------------+-----------+



In [97]:
spark_new_app_session.sql('show tables in global_temp').show()

+-----------+---------------+-----------+
|  namespace|      tableName|isTemporary|
+-----------+---------------+-----------+
|global_temp|temp_customeers|       true|
+-----------+---------------+-----------+



In [98]:
spark_new_app_session.sql('show tables').show()

+---------+--------------------+-----------+
|namespace|           tableName|isTemporary|
+---------+--------------------+-----------+
|  default|customers_persistent|      false|
+---------+--------------------+-----------+



In [99]:
spark_new_app_session.sql('select* from customers_persistent limit 5').show()

+-----------+----------+---------+-----------+-------+-----------------+---------+
|customer_id|      name|     city|      state|country|registration_date|is_active|
+-----------+----------+---------+-----------+-------+-----------------+---------+
|          0|Customer_0|     Pune|Maharashtra|  India|       2023-06-29|    false|
|          1|Customer_1|Bangalore| Tamil Nadu|  India|       2023-12-07|     true|
|          2|Customer_2|Hyderabad|    Gujarat|  India|       2023-10-27|     true|
|          3|Customer_3|Bangalore|  Karnataka|  India|       2023-10-17|    false|
|          4|Customer_4|Ahmedabad|  Karnataka|  India|       2023-03-14|    false|
+-----------+----------+---------+-----------+-------+-----------------+---------+



In [100]:
spark_new_app_session.sql('describe extended customers_persistent').show(truncate=False)

+----------------------------+--------------------------------------------------------------+-------+
|col_name                    |data_type                                                     |comment|
+----------------------------+--------------------------------------------------------------+-------+
|customer_id                 |int                                                           |NULL   |
|name                        |string                                                        |NULL   |
|city                        |string                                                        |NULL   |
|state                       |string                                                        |NULL   |
|country                     |string                                                        |NULL   |
|registration_date           |date                                                          |NULL   |
|is_active                   |boolean                                             

In [90]:
spark.sparkContext.applicationId

'application_1744795247389_0001'

In [91]:
spark_new.sparkContext.applicationId

'application_1744795247389_0001'